# ML Experiment 1 - HSE CC PQ Incidents Prediction (June '25)

## Load Libraries

In [0]:
!python --version

Python 3.12.3


In [0]:
!python -m pip install --upgrade pip
!python -m pip install autogluon imbalanced-learn loguru
!pip install -U ray[data,train,tune,serve]==2.39.0
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install -U fastai==2.7.19

  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
DEPRECATION: Using the pkg_resources metadata backend is deprecated. pip 26.3 will enforce this behaviour change. A possible replacement is to use the default importlib.metadata backend, by unsetting the _PIP_USE_IMPORTLIB_METADATA environment variable. Discussion can be found at https://github.com/pypa/pip/issues/13317
  Using cached autogluon-1.3.1-py3-none-any.whl.metadata (11 kB)
  Using cached imbalanced_learn-0.13.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
  Using cached autogluon.core-1.3.1-py3-none-any.whl.metadata (12 kB)
  Using cached autogluon.features-1.3.1-py3-none-any.whl.metadata (11 kB)
  Using cached autogluon.tabular-1.3.1-py3-none-any.whl.metadata (14 kB)
  Using cach

In [0]:
dbutils.library.restartPython()

In [0]:
import mlflow; mlflow.autolog(disable=True)

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from autogluon.tabular import TabularDataset, TabularPredictor

from imblearn.over_sampling import SMOTE

from loguru import logger
from collections import Counter

## Classification with SMOTE

In [0]:
# Load ML dataset
ml_df_class = spark.table('hive_metastore.default.hse_cc_pred_ml').toPandas().drop(columns=['number_of_near_miss_incidents'])

# Create training and test datasets
label = 'is_near_miss'
keys = ['production_date']
X = ml_df_class.drop(columns=[label]+keys)
y = ml_df_class[label]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

# Use standard scaling
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# Solve class imbalance using SMOTE
# logger.info(f'Class distribution before SMOTE: {Counter(y_train)}')
# sm = SMOTE(random_state=42)
# X_train, y_train = sm.fit_resample(X_train, y_train)
# logger.info(f'Class distribution after SMOTE: {Counter(y_train)}')

# Prepare the datasets for Autogluon
train_data = pd.DataFrame(X_train, columns=X.columns)
train_data[label] = y_train.values

test_data = pd.DataFrame(X_test, columns=X.columns)
test_data[label] = y_test.values

train_data_auto = TabularDataset(train_data)
test_data_auto = TabularDataset(test_data)

logger.info(f"Train data shape: {train_data_auto.shape}")
logger.info(f"Test data shape: {test_data_auto.shape}")

2025-06-23 06:05:11.781 | INFO     | __main__:<module>:32 - Train data shape: (1534, 26)
2025-06-23 06:05:11.782 | INFO     | __main__:<module>:33 - Test data shape: (658, 26)


In [0]:
# label = 'is_near_miss'
# train_data_auto = spark.table('hive_metastore.groupdb_rakona.hse_cc_inc_pred_ml_train').toPandas()
# test_data_auto = spark.table('hive_metastore.groupdb_rakona.hse_cc_inc_pred_ml_test').toPandas()

# print(f"Train data shape: {train_data_auto.shape}")
# print(f"Test data shape: {test_data_auto.shape}")

In [0]:
predictor = TabularPredictor(label=label, eval_metric="f1").fit(
    train_data_auto,
    # time_limit=600,
    presets="best_quality"
    # num_bag_folds=5,
    # num_stack_levels=2
)

No path specified. Models will be saved in: "AutogluonModels/ag-20250623_060512"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.12.3
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #33~22.04.1-Ubuntu SMP Fri Apr 25 06:39:10 UTC 2025
CPU Count:          8
Memory Avail:       39.92 GB / 57.39 GB (69.6%)
Disk Space Avail:   10.00 GB / 10.00 GB (100.0%)
	We recommend a minimum available disk space of 10 GB, and large datasets may require more.
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.


In [0]:
predictor.leaderboard(test_data_auto, extra_metrics=['accuracy', 'precision', 'recall'])

/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/databricks/python/lib/python3.12/site-packages/s

,model,score_test,accuracy,precision,recall,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_r50_BAG_L1,0.125000,0.978723,0.333333,0.076923,0.108108,f1,1.258348,0.059675,4.643626,1.258348,0.059675,4.643626,1,True,28
1,NeuralNetFastAI_r191_BAG_L2,0.125000,0.957447,0.105263,0.153846,0.634921,f1,22.920179,1.779616,113.941060,2.318692,0.178860,14.556843,2,True,103
2,RandomForest_r166_BAG_L1,0.117647,0.977204,0.250000,0.076923,0.111111,f1,0.355407,0.059912,0.985944,0.355407,0.059912,0.985944,1,True,63
3,RandomForestGini_BAG_L1,0.117647,0.977204,0.250000,0.076923,0.111111,f1,0.363959,0.098715,0.826233,0.363959,0.098715,0.826233,1,True,5
4,RandomForestEntr_BAG_L1,0.117647,0.977204,0.250000,0.076923,0.200000,f1,0.406090,0.120731,0.910077,0.406090,0.120731,0.910077,1,True,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,NeuralNetFastAI_r102_BAG_L2,0.000000,0.963526,0.000000,0.000000,0.625000,f1,23.110988,1.861397,115.459364,2.509501,0.260641,16.075147,2,True,109
138,XGBoost_r33_BAG_L2,0.000000,0.975684,0.000000,0.000000,0.418605,f1,23.350826,1.653194,103.202182,2.749339,0.052438,3.817965,2,True,106
139,NeuralNetFastAI_r11_BAG_L2,0.000000,0.968085,0.000000,0.000000,0.703704,f1,23.372835,1.836552,114.474643,2.771348,0.235796,15.090425,2,True,117
140,NeuralNetFastAI_r134_BAG_L2,0.000000,0.968085,0.000000,0.000000,0.703704,f1,23.559485,1.769566,134.918080,2.957998,0.168810,35.533863,2,True,138


In [0]:
predictor.evaluate(test_data_auto, model='CatBoost_r50_BAG_L1', silent=True)

{'f1': np.float64(0.125),
 'accuracy': 0.9787234042553191,
 'balanced_accuracy': np.float64(0.5369111508646393),
 'mcc': np.float64(0.15249578439995462),
 'roc_auc': np.float64(0.923494335122242),
 'precision': np.float64(0.3333333333333333),
 'recall': np.float64(0.07692307692307693)}

In [0]:
predictor.feature_importance(test_data_auto, model='CatBoost_r50_BAG_L1')

These features in provided data are not utilized by the predictor and will be ignored: ['breakdowns']
Computing feature importance via permutation shuffling for 24 features using 658 rows with 5 shuffle sets...
	93.33s	= Expected runtime (18.67s per shuffle set)
	1.84s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
department_name,0.125000,0.000000,0.500000,5,0.125000,0.125000
line_name,0.125000,0.000000,0.500000,5,0.125000,0.125000
constrained_stops,0.125000,0.000000,0.500000,5,0.125000,0.125000
planned_count,0.125000,0.000000,0.500000,5,0.125000,0.125000
unplanned_count,0.125000,0.000000,0.500000,5,0.125000,0.125000
planned_stops_per_day,0.125000,0.000000,0.500000,5,0.125000,0.125000
total_defects_yesterday,0.125000,0.000000,0.500000,5,0.125000,0.125000
unplanned_stops_per_day,0.125000,0.000000,0.500000,5,0.125000,0.125000
safe_percentage,0.125000,0.000000,0.500000,5,0.125000,0.125000
per_resolved_in_24_hours,0.125000,0.000000,0.500000,5,0.125000,0.125000
